# UpdateManagerEntraID

### This notebook shows the step by step of using Powershell Graph SDK (Set-MgUserManagerByRef) to edit/ update the manager of EntraID user.

### First it will show how to export the EntraID user properties from a fiction tenant. Then, it will show how to build a user to object id and a object id to user hash. It will use the hash to record the initial manager status. 

### After that, it will show setting up the table structure and update the user's manager.

## Export user to csv

In [1]:
# export the EntraID user into users.csv

![export](./exportUsers.jpg)

In [2]:
# assume the filename is users.csv

$u = Import-Csv './users.csv'

In [3]:
$userToIdHash = @{}
$idToUserHash = @{}

$u | ForEach-Object -process {
   $t = @{$_.'Display name' = $_.'Object Id'}
   $userToIdHash += $t

   $t = @{$_.'Object Id' = $_.'Display name'}
   $idToUserHash += $t
}

In [4]:
$userToIdHash


Name                           Value
----                           -----
Patti Fernandez                ac4e7475-bf36-4ac3-b8bd-5e0bb75a630d
Joni Sherman                   96ba2e29-c833-4322-83e1-1f609aa3fe4c
Megan Bowen                    c948120e-0b75-4e47-b758-c6c57bddde64
Alex Wilber                    c3a3bf8e-d9d4-4b9a-b7d4-175993bbe726
Grady Archie                   2ea73151-a011-47b5-9a99-8d37adc221b5
Diego Siciliani                0a02a0f1-48e1-4010-b904-c7650b34c112
Pradeep Gupta                  9091d05e-e280-4b72-8612-fd8e76ae4795
Johanna Lorenz                 40d46eb7-4a18-4fa0-b410-c40c92324624
Lynne Robbins                  27baf846-be3a-4bc0-b382-40d466d82e1b
Daniel Carmack                 9a2e7c71-2c59-4b80-ada4-19d8534e6d99
P A                            b28254f0-80be-45dd-a1f4-d954e1e1a5f3
Henrietta Mueller              2c758750-2644-4e13-975f-453e0dfa2263
Adele Vance                    e065d899-0464-4a43-b02a-206028b27982
Nestor Wilke                   33d9a4ec-2

In [5]:
$idToUserHash


Name                           Value
----                           -----
27baf846-be3a-4bc0-b382-40d46… Lynne Robbins
0e799665-9e0b-4226-97c3-694a4… Miriam Graham
9a2e7c71-2c59-4b80-ada4-19d85… Daniel Carmack
40d46eb7-4a18-4fa0-b410-c40c9… Johanna Lorenz
33d9a4ec-2426-4c08-b4f3-97459… Nestor Wilke
9091d05e-e280-4b72-8612-fd8e7… Pradeep Gupta
c3a3bf8e-d9d4-4b9a-b7d4-17599… Alex Wilber
e065d899-0464-4a43-b02a-20602… Adele Vance
a0df3ed6-b117-4faa-aedf-ab78d… Lidia Holloway
2c758750-2644-4e13-975f-453e0… Henrietta Mueller
f243a1ce-a5e6-42e1-abe1-f49ca… Isaiah Langer
96ba2e29-c833-4322-83e1-1f609… Joni Sherman
0a02a0f1-48e1-4010-b904-c7650… Diego Siciliani
ac4e7475-bf36-4ac3-b8bd-5e0bb… Patti Fernandez
2ea73151-a011-47b5-9a99-8d37a… Grady Archie
b28254f0-80be-45dd-a1f4-d954e… P A
c948120e-0b75-4e47-b758-c6c57… Megan Bowen
98d68316-30d8-43c4-ac35-4319e… Lee Gu



## Record the manager status of the tenant 


In [6]:
Connect-MgGraph -Scopes "User.ReadWrite.All"

Welcome to Microsoft Graph!

Connected via delegated access using 14d82eec-204b-4c2f-b7e8-296a70dab67e
Readme: https://aka.ms/graph/sdk/powershell
SDK Docs: https://aka.ms/graph/sdk/powershell/docs
API Docs: https://aka.ms/graph/docs

NOTE: You can use the -NoWelcome parameter to suppress this message.



In [7]:
$users = Get-MgUser

In [8]:
$users | ForEach-Object -Begin {$userManagerArr = @()} `
                        -Process {
                                    $userManagerId = Get-MgUserManager -UserId $_.Id -ErrorAction SilentlyContinue
                                    if ($userManagerId) {
                                        $userManager = $idToUserHash[$userManagerId.Id]
                                    }
                                    else {
                                        $userManager = 'No Manager'
                                    }
                                    $userManagerArr += [PSCustomObject] @{Name = $_.DisplayName;
                                                                            Manager = $userManager;
                                                                            ManagerId = $userManagerId.Id}
                                }

                        

Error: Command failed: SubmitCode: $users | ForEach-Object -Begin {$userManagerArr =  ...

In [9]:
$userManagerArr


Name              Manager         ManagerId
----              -------         ---------
Adele Vance       Miriam Graham   0e799665-9e0b-4226-97c3-694a49c45c63
Alex Wilber       Miriam Graham   0e799665-9e0b-4226-97c3-694a49c45c63
Daniel Carmack    No Manager      
Diego Siciliani   Nestor Wilke    33d9a4ec-2426-4c08-b4f3-97459e49cd3b
Grady Archie      Lee Gu          98d68316-30d8-43c4-ac35-4319ec3b50e9
Henrietta Mueller Lee Gu          98d68316-30d8-43c4-ac35-4319ec3b50e9
Isaiah Langer     Miriam Graham   0e799665-9e0b-4226-97c3-694a49c45c63
Johanna Lorenz    Lee Gu          98d68316-30d8-43c4-ac35-4319ec3b50e9
Joni Sherman      Nestor Wilke    33d9a4ec-2426-4c08-b4f3-97459e49cd3b
Lee Gu            Patti Fernandez ac4e7475-bf36-4ac3-b8bd-5e0bb75a630d
Lidia Holloway    Lee Gu          98d68316-30d8-43c4-ac35-4319ec3b50e9
Lynne Robbins     Miriam Graham   0e799665-9e0b-4226-97c3-694a49c45c63
Megan Bowen       Miriam Graham   0e799665-9e0b-4226-97c3-694a49c45c63
Miriam Graham     Patti 

In [10]:
# export it to csv so we can restore the manager info later

$userManagerArr | Export-Csv -Path './usermanager.csv' 


## Setup the table to prepare to update the user's manager

In [11]:
# use the exported EntraID user data (users.csv) above and add a column ToManager. ToManager is a 
# dropdown list which contains all the users in the organization then assign new manager to the user. 
# After that, save the file into users.xlsx

![toManager](./toManager.jpg)

In [12]:
# we need a cmdlet from module ImportExcel, install the module by
# Install-Module -Name ImportExcel

In [13]:
$u = Import-Excel -Path "./users.xlsx"

In [14]:
# display the user and toManager table 

$u| Select-Object @{Label='Name'; Expression={$_.'Display Name'}}, 
                    @{Label='ID'; Expression={$_.'Object Id'}}, 
                    @{Label='ToManager'; Expression={$_.ToManager}},
                    @{Label='ToManager ID'; Expression={$userToIdHash[$_.ToManager]}}


Name              ID                                   ToManager       ToManager ID
----              --                                   ---------       ------------
Adele Vance       e065d899-0464-4a43-b02a-206028b27982 Patti Fernandez ac4e7475-bf36-4ac3-b8bd-5e0…
Alex Wilber       c3a3bf8e-d9d4-4b9a-b7d4-175993bbe726 Pradeep Gupta   9091d05e-e280-4b72-8612-fd8…
Daniel Carmack    9a2e7c71-2c59-4b80-ada4-19d8534e6d99 Patti Fernandez ac4e7475-bf36-4ac3-b8bd-5e0…
Diego Siciliani   0a02a0f1-48e1-4010-b904-c7650b34c112 Pradeep Gupta   9091d05e-e280-4b72-8612-fd8…
Grady Archie      2ea73151-a011-47b5-9a99-8d37adc221b5 Patti Fernandez ac4e7475-bf36-4ac3-b8bd-5e0…
Henrietta Mueller 2c758750-2644-4e13-975f-453e0dfa2263 Pradeep Gupta   9091d05e-e280-4b72-8612-fd8…
Isaiah Langer     f243a1ce-a5e6-42e1-abe1-f49ca450d6a0 Patti Fernandez ac4e7475-bf36-4ac3-b8bd-5e0…
Johanna Lorenz    40d46eb7-4a18-4fa0-b410-c40c92324624 Pradeep Gupta   9091d05e-e280-4b72-8612-fd8…
Joni Sherman      96ba2e29-c833

Error: Command failed: SubmitCode: # display the user and toManager table  ...

## Use Set-MgUserManagerByRef to update user's manager

In [15]:
$newManagerTable = $u | Select-Object @{Label='Name'; Expression={$_.'Display Name'}}, 
                        @{Label='ID'; Expression={$_.'Object Id'}}, 
                        @{Label='ToManager'; Expression={$_.ToManager}},
                        @{Label='ToManager ID'; Expression={$userToIdHash[$_.ToManager]}}

Error: Command failed: SubmitCode: $newManagerTable = $u | Select-Object @{Label='Nam ...

In [16]:
$newManagerTable | ForEach-Object {
    if ($_.'ToManager ID') {
        $odataId = "https://graph.microsoft.com/v1.0/users/{0}" -f $_."ToManager ID"
        "Updating user {0} manager to {1}, objectId: {2}" -f $_.Name, $_.ToManager, $_.'ToManager ID'
        $odataId

        # update user's manager
        $NewManager = @{
            "@odata.id" = $odataId
        }
        Set-MgUserManagerByRef -UserId $_.ID -BodyParameter $NewManager
    }
    else {
        "Removing user {0}'s manager" -f $_.Name
        Remove-MgUserManagerByRef -UserId $_.ID
    }
}

Updating user Adele Vance manager to Patti Fernandez, objectId: ac4e7475-bf36-4ac3-b8bd-5e0bb75a630d
https://graph.microsoft.com/v1.0/users/ac4e7475-bf36-4ac3-b8bd-5e0bb75a630d
Updating user Alex Wilber manager to Pradeep Gupta, objectId: 9091d05e-e280-4b72-8612-fd8e76ae4795
https://graph.microsoft.com/v1.0/users/9091d05e-e280-4b72-8612-fd8e76ae4795
Updating user Daniel Carmack manager to Patti Fernandez, objectId: ac4e7475-bf36-4ac3-b8bd-5e0bb75a630d
https://graph.microsoft.com/v1.0/users/ac4e7475-bf36-4ac3-b8bd-5e0bb75a630d
Updating user Diego Siciliani manager to Pradeep Gupta, objectId: 9091d05e-e280-4b72-8612-fd8e76ae4795
https://graph.microsoft.com/v1.0/users/9091d05e-e280-4b72-8612-fd8e76ae4795
Updating user Grady Archie manager to Patti Fernandez, objectId: ac4e7475-bf36-4ac3-b8bd-5e0bb75a630d
https://graph.microsoft.com/v1.0/users/ac4e7475-bf36-4ac3-b8bd-5e0bb75a630d
Updating user Henrietta Mueller manager to Pradeep Gupta, objectId: 9091d05e-e280-4b72-8612-fd8e76ae4795
https:

## Check user's manager after the update

In [17]:
$users = Get-MgUser

In [18]:
$users | ForEach-Object {
                $managerId = Get-MgUserManager -UserId $_.Id -ErrorAction SilentlyContinue
                if ($managerId) {
                    "{0}'s manager is {1}" -f $_.DisplayName, $idToUserHash[$managerId.Id]
                }
                else {
                    "{0} has no manager" -f $_.DisplayName
                }

        }

Adele Vance's manager is Patti Fernandez
Alex Wilber's manager is Pradeep Gupta
Daniel Carmack's manager is Patti Fernandez
Diego Siciliani's manager is Pradeep Gupta
Grady Archie's manager is Patti Fernandez
Henrietta Mueller's manager is Pradeep Gupta
Isaiah Langer's manager is Patti Fernandez
Johanna Lorenz's manager is Pradeep Gupta
Joni Sherman's manager is Patti Fernandez
Lee Gu's manager is Pradeep Gupta
Lidia Holloway's manager is Patti Fernandez
Lynne Robbins's manager is Pradeep Gupta
Megan Bowen's manager is Patti Fernandez
Miriam Graham's manager is Pradeep Gupta
Nestor Wilke's manager is Patti Fernandez
Patti Fernandez's manager is Pradeep Gupta
P A's manager is Pradeep Gupta
Pradeep Gupta has no manager


Error: Command failed: SubmitCode: $users | ForEach-Object { ...